In [0]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


https://appliedmachinelearning.blog/2017/06/14/voice-gender-detection-using-gmms-a-python-primer/

In [3]:
! wget https://www.dropbox.com/s/f8e4ax4nvp2p0fe/pygender.zip?dl=0

--2020-05-15 04:06:11--  https://www.dropbox.com/s/f8e4ax4nvp2p0fe/pygender.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/f8e4ax4nvp2p0fe/pygender.zip [following]
--2020-05-15 04:06:12--  https://www.dropbox.com/s/raw/f8e4ax4nvp2p0fe/pygender.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7570d85ab9062828935ecf9b7c.dl.dropboxusercontent.com/cd/0/inline/A3tS0khi-WuS20Ljx3CaGYKEgfYPWzkn4b7DMZKMg2-K82NJdp0hn9P_hY1W9nmPDwcRcfnqqmHxCGdVB9kH3sC0NFxKuGxJltreo6kPlCWM0HeFOdyuYV-VNo7v15pGLn0/file# [following]
--2020-05-15 04:06:12--  https://uc7570d85ab9062828935ecf9b7c.dl.dropboxusercontent.com/cd/0/inline/A3tS0khi-WuS20Ljx3CaGYKEgfYPWzkn4b7DMZKMg2-K82NJdp0hn9P_hY1W9nmPDwcRcfnqqmHxCGdVB9kH3sC0NFxKuGxJltreo6kPlCWM0

In [4]:
! unzip '/content/pygender.zip'

Archive:  /content/pygender.zip
   creating: pygender/test_data/
   creating: pygender/test_data/AudioSet/
   creating: pygender/test_data/AudioSet/female_clips/
  inflating: pygender/test_data/AudioSet/female_clips/-_m_h0DBaxw.wav  
  inflating: pygender/test_data/AudioSet/female_clips/-_TeUwS-1Hs.wav  
  inflating: pygender/test_data/AudioSet/female_clips/-_wc9AWT4ys.wav  
  inflating: pygender/test_data/AudioSet/female_clips/00BG6yL5zGw.wav  
  inflating: pygender/test_data/AudioSet/female_clips/014i-70HC_s.wav  
  inflating: pygender/test_data/AudioSet/female_clips/01Q0Se7F28Y.wav  
  inflating: pygender/test_data/AudioSet/female_clips/038H9i5KHlM.wav  
  inflating: pygender/test_data/AudioSet/female_clips/03FIC_TqMQg.wav  
  inflating: pygender/test_data/AudioSet/female_clips/03rU2dEkVsM.wav  
  inflating: pygender/test_data/AudioSet/female_clips/04DeW1O4fm4.wav  
  inflating: pygender/test_data/AudioSet/female_clips/04e9N_422v0.wav  
  inflating: pygender/test_data/AudioSet/femal

# Voice Gender Detection using GMMs : A Python Primer

The below data-sets can be downloaded from here.

  Training corpus : It has been developed from YouTube videos and consists of 5 minutes of speech for each gender, spoken by 5 distinct male and 5 female speakers (i.e, 1 minute/speaker).
  
  Test corpus: It has been extracted from “AudioSet“, a large scale manually annotated corpus recently released by Google this year (2017). The subset constructed from it contains 558 female only speech utterances and 546 male only speech utterances. All audio files are of 10 seconds duration and are sampled at 16000 Hz.


We will give a brief primer about how to work with speech signals. From the speech signals in training data, a popular speech feature, Mel Frequency Cepstrum Coefficients (MFCCs), will be extracted; they are known to contain gender information (among other things). The 2 gender models are built by using yet another famous ML technique – Gaussian Mixture Models (GMMs). A GMM will take as input the MFCCs of the training samples and will try to learn their distribution, which will be representative of the gender. Now, when the gender of a new voice sample is to be detected, first the MFCCs of the sample will be extracted and then the trained GMM models will be used to calculate the scores of the features for both the models. Model with the maximum score is predicted as gender of the test speech. Having given an overview of the approach, presented following is the organisation of this blog post:

# 1. Working with speech frames

A speech signal is just a sequence of numbers which denote the amplitude of the speech spoken by the speaker. We need to understand 3 core concepts while working with speech signals:

  Framing – Since speech is a non-stationary signal, its frequency contents are continuously changing with time. In order to do any sort of analysis of the signal, such as knowing its frequency contents for short time intervals (known as Short Term Fourier Transform of the signal), we need to be able to view it as a stationary signal. To achieve this stationarity, the speech signal is divided into short frames of duration 20 to 30 milliseconds, as the shape of our vocal tract can be assumed to be unvarying for such small intervals of time. Frames shorter than this duration won’t have enough samples to give a good estimate of the frequency components, while in longer frames the signal may change too much within the frame that the condition of stationary no more holds.

Windowing – Extracting raw frames from a speech signal can lead to discontinuities towards the endpoints due to non-integer number of periods in the extracted waveform, which will then lead to an erroneous frequency representation (known as spectral leakage in signal processing lingo). This is prevented by multiplying a window function with the speech frame. A window function’s amplitude gradually falls to zero towards its two end and thus this multiplication minimizes the amplitude of the above mentioned discontinuities.

Overlapping frames – Due to windowing, we are actually losing the samples towards the beginning and the end of the frame; this too will lead to an incorrect frequency representation. To compensate for this loss, we take overlapping frames rather than disjoint frames, so that the samples lost from the end of the ith frame and the beginning of the (i+1)th frame are wholly included in the frame formed by the overlap between these 2 frames. The overlap between frames is generally taken to be of 10-15 ms.

# 2. Extracting MFCC features

Having extracted the speech frames, we now proceed to derive MFCC features for each speech frame.  Speech is produced by humans by filtering applied by our vocal tract on the air expelled by our lungs. The properties of the source (lungs) are common for all speakers; it is the properties of the vocal tract, which is responsible for giving shape to the spectrum of signal and it varies across speakers. The shape of the vocal tract governs what sound is produced and the MFCCs best represent this shape.

MFCCs are mel-frequency cepstral coefficients which are some transformed values of signal in cepstral domain. From theory of speech production, speech is assumed to be convolution of source (air expelled from lungs) and filter (our vocal tract). The purpose here is to characterise the filter and remove the source part. In order to achieve this,


  We first transform the time domain speech signal into spectral domain signal using Fourier transform where source and filter part are now in multiplication.

  Take log of the transformed values so that source and filter are now additive in log spectral domain. Use of log to transform from multiplication to summation made it easy to separate source and filter using a linear filter.

  Finally, we apply discrete cosine transform (found to be more successful than FFT or I-FFT) of the log spectral signal to get MFCCs. Initially the idea was to transform the log spectral signal to time domain using Inverse-FFT but ‘log’ being a non-linear operation created new frequencies called Quefrency or say it transformed the log spectral signal into a new domain called cepstral domain (ceps being reverse of spec).
  
  The reason for the term ‘mel’ in MFFC is mel scale which exactly specifies how to space our frequency regions. Humans are much better at discerning small changes in pitch at low frequencies than they are at high frequencies. Incorporating this scale makes our features match more closely what humans hear.

The above explanation is just to give the readers an idea of how these features were motivated. If you really like to explore and understand more about MFCCs, you can read in this blog. You can find various implementation for extracting MFCCs on internet. We have employed python_speech_features (check here). All you have to do is pip install python_speech_features. One can also install scikits talkbox for the same. The following python code is a function to extract MFCC features from given audio.

In [6]:
! pip install cPickle

ERROR: Could not find a version that satisfies the requirement cPickle (from versions: none)
ERROR: No matching distribution found for cPickle


In [0]:
import python_speech_features as mfcc
def get_MFCC(sr,audio):
    features = mfcc.mfcc(audio, sr, 0.025, 0.01, 13, appendEnergy = False)
    features = preprocessing.scale(features)
    return features

# 3. Training gender models

In order to build a gender detection system from the above extracted features, we need to model both the genders. We employ GMMs for this task.

A Gaussian mixture model is a probabilistic clustering model for representing the presence of sub-populations within an overall population.  The idea of training a GMM is to approximate the probability distribution of a class by a linear combination of ‘k’ Gaussian distributions/clusters, also called the components of the GMM. The likelihood of data points (feature vectors) for a model is given by following equation:

Python’s sklearn.mixture package is used by us to learn a GMM from the features matrix containing the MFCC features. The GMM object requires the number of components n_components to be fitted on the data, the number of iterations n_iter to be performed for estimating the parameters of these n components, the type of co-variance covariance_type to be assumed between the features and the number of times n_ init the K-means initialization is to be done. The initialization which gave the best results is kept. The fit() function then estimates the model parameters using the EM algorithm.

The following Python code is used to train the gender models. The code is run once for each gender and source is given the path to the training files for the respective gender.

In [0]:
from sklearn.mixture import GMM
gmm = GMM(n_components = 8, n_iter = 200, covariance_type='diag',n_init = 3)
gmm.fit(features)

In [17]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture as GMM
import python_speech_features as mfcc
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")
 
def get_MFCC(sr,audio):
    features = mfcc.mfcc(audio,sr, 0.025, 0.01, 13,appendEnergy = False)
    features = preprocessing.scale(features)
    return features
 
#path to training data
source   = "/content/pygender/train_data/youtube/male"
#path to save trained model
dest     = "/content/pygender"
files    = [os.path.join(source,f) for f in os.listdir(source) if
             f.endswith('.wav')]
features = np.asarray(());
 
for f in files:
    sr,audio = read(f)
    vector   = get_MFCC(sr,audio)
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
 
gmm = GMM(n_components = 8, covariance_type='diag',
        n_init = 3)
gmm.fit(features)
picklefile = f.split("/")[-2].split(".wav")[0]+".gmm"
 
# model saved as male.gmm
filename = '/content/pygender/Male.gmm'
outfile = open(filename,'wb')
pickle.dump(gmm, outfile)
outfile.close()
print('modeling completed for gender:',picklefile)

modeling completed for gender: male.gmm


In [18]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture as GMM
import python_speech_features as mfcc
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")
 
def get_MFCC(sr,audio):
    features = mfcc.mfcc(audio,sr, 0.025, 0.01, 13,appendEnergy = False)
    features = preprocessing.scale(features)
    return features
 
#path to training data
source   = "/content/pygender/train_data/youtube/female"
#path to save trained model
dest     = "/content/pygender"
files    = [os.path.join(source,f) for f in os.listdir(source) if
             f.endswith('.wav')]
features = np.asarray(());
 
for f in files:
    sr,audio = read(f)
    vector   = get_MFCC(sr,audio)
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
 
gmm = GMM(n_components = 8, covariance_type='diag',
        n_init = 3)
gmm.fit(features)
picklefile = f.split("/")[-2].split(".wav")[0]+".gmm"
 
# model saved as male.gmm
filename = '/content/pygender/Female.gmm'
outfile = open(filename,'wb')
pickle.dump(gmm, outfile)
outfile.close()
print('modeling completed for gender:',picklefile)

modeling completed for gender: female.gmm


# 4. Evaluation on subset of AudioSet corpus

Upon arrival of a test voice sample for gender detection, we begin by extracting the MFCC features for it, with 25 ms frame size and 10 ms overlap between frames . Next we require the log likelihood scores for each frame of the sample, x_1, x_2, ... ,x_i , belonging to each gender, ie, P(x_i|female) and P(x_i|male) is to be calculated. Using (2), the likelihood of the frame being from a female voice is calculated by substituting the \mu and \Sigma of female GMM model. This is done for each of the k Gaussian components in the model, and the weighted  sum of the k likelihoods from the components is taken as per the w parameter of the model, just like in (1). The logarithm operation when applied on the obtained sum gives us the log likelihood value for the frame. This is repeated for all the frames of the sample and the likelihoods of all the frames are added.

Similar to this, the likelihood of the speech being male is calculated by substituting the values of the parameters of the trained male GMM model and repeating the above procedure for all the frames. The Python code given below predicts the gender of the test audio.

In [26]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
import python_speech_features as mfcc
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")
def get_MFCC(sr,audio):
    features = mfcc.mfcc(audio,sr, 0.025, 0.01, 13,appendEnergy = False)
    feat     = np.asarray(())
    for i in range(features.shape[0]):
        temp = features[i,:]
        if np.isnan(np.min(temp)):
            continue
        else:
            if feat.size == 0:
                feat = temp
            else:
                feat = np.vstack((feat, temp))
    features = feat;
    features = preprocessing.scale(features)
    return features
 
#path to test data
sourcepath = "/content/pygender/test_data/AudioSet/female_clips"
#path to saved models
modelpath  = "/content/pygender"    
 
gmm_files = [os.path.join(modelpath,fname) for fname in
              os.listdir(modelpath) if fname.endswith('.gmm')]
models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
genders   = [fname.split("/")[-1].split(".gmm")[0] for fname
              in gmm_files]
files     = [os.path.join(sourcepath,f) for f in os.listdir(sourcepath)
              if f.endswith(".wav")] 
total_file=0
correct=0
wrong=0
for f in files:
    #print(f.split("/")[-1])
    total_file+=1
    sr, audio  = read(f)
    features   = get_MFCC(sr,audio)
    scores     = None
    log_likelihood = np.zeros(len(models))
    for i in range(len(models)):
        gmm    = models[i]         #checking with each model one by one
        scores = np.array(gmm.score(features))
        log_likelihood[i] = scores.sum()
    winner = np.argmax(log_likelihood)
    if(genders[winner]=='Female'):
      correct+=1
    else:
      wrong+=1
    #print("\tdetected as - ", genders[winner],"\n\tscores:female ",log_likelihood[0],",male ", log_likelihood[1],"\n")
print("total no of files = ",total_file)
print("Correctly identified files = ", correct)
print("Wrong identification = ", wrong)

total no of files =  558
Correctly identified files =  531
Wrong identification =  27


In [27]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
import python_speech_features as mfcc
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")
def get_MFCC(sr,audio):
    features = mfcc.mfcc(audio,sr, 0.025, 0.01, 13,appendEnergy = False)
    feat     = np.asarray(())
    for i in range(features.shape[0]):
        temp = features[i,:]
        if np.isnan(np.min(temp)):
            continue
        else:
            if feat.size == 0:
                feat = temp
            else:
                feat = np.vstack((feat, temp))
    features = feat;
    features = preprocessing.scale(features)
    return features
 
#path to test data
sourcepath = "/content/pygender/test_data/AudioSet/male_clips"
#path to saved models
modelpath  = "/content/pygender"    
 
gmm_files = [os.path.join(modelpath,fname) for fname in
              os.listdir(modelpath) if fname.endswith('.gmm')]
models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
genders   = [fname.split("/")[-1].split(".gmm")[0] for fname
              in gmm_files]
files     = [os.path.join(sourcepath,f) for f in os.listdir(sourcepath)
              if f.endswith(".wav")] 
total_file=0
correct=0
wrong=0
for f in files:
    #print(f.split("/")[-1])
    total_file+=1
    sr, audio  = read(f)
    features   = get_MFCC(sr,audio)
    scores     = None
    log_likelihood = np.zeros(len(models))
    for i in range(len(models)):
        gmm    = models[i]         #checking with each model one by one
        scores = np.array(gmm.score(features))
        log_likelihood[i] = scores.sum()
    winner = np.argmax(log_likelihood)
    if(genders[winner]=='Male'):
      correct+=1
    else:
      wrong+=1
    #print("\tdetected as - ", genders[winner],"\n\tscores:female ",log_likelihood[0],",male ", log_likelihood[1],"\n")
print("total no of files = ",total_file)
print("Correctly identified files = ", correct)
print("Wrong identification = ", wrong)

total no of files =  546
Correctly identified files =  414
Wrong identification =  132


http://www.primaryobjects.com/2016/06/22/identifying-the-gender-of-a-voice-using-machine-learning/

Concluding Remarks

We hope the blog post was successful in explaining to you your first speech processing task. We expect you to reproduce the results posted by us. In order to get a better understanding of the various parameters used by us, you can play with them, for example:

  Train gender models on larger data-set. In the experiments performed, training data consists of  5 minutes of speech per gender only. A larger data-set may improve the accuracy as it will encompass the MFCCs well.

  Clean the extracted data-set from AudioSet. We have not done any cleaning or noise removal. Few of the audios are noisy and the spoken speech part in audio is less in the 10 seconds audio clip.

  Readers can study the effect of number of GMM components on the performance of the models.

  We have taken only MFCCs in order to build a gender detection system. In literature, you can find lot of acoustic features that are used to build gender model. For reference here is a link.


In [0]:
a=10

# Spoken Speaker Identification based on Gaussian Mixture Models : Python Implementation

Similar to our previous post “Voice Gender Detection“, this blog-post focuses on a beginner’s method to answer the question ‘who is the speaker‘ in the speech file. Recently, lot of voice biometric systems have been developed which can extract speaker information from the recorded voice and identify the speaker from set of trained speakers in the database. In this blog-post, we will illustrate the same with a naive approach using Gaussian Mixture Models (GMM). There are other conventional as well as modern approaches which are more robust to channel noise and also performs better than approach followed in this blog-post.

    GMM-UBM (Gaussian Mixture Model – Universal Background Model) using MAP (Maximum Aposteriori) adaptation [1] is one of the successful conventional technique to implement speaker identification.
    I-vectors based speaker identification [2] is the state-of-the-art technique implemented in lot of voice biometric products.

As a beginner, the above mentioned techniques may overwhelm you as they are mathematically complex methods and requires some research effort in order to comprehend. Therefore, I am not following any of the two approaches. Instead, I am interested in showing you the implementation of fundamental step of speaker identification (using GMMs) which can then lead to development of GMM-UBM or I-vectors approach.

In [29]:
! wget https://www.dropbox.com/s/k89d0cd0dfeikh6/development_set.zip?dl=0

--2020-05-15 05:13:09--  https://www.dropbox.com/s/k89d0cd0dfeikh6/development_set.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/k89d0cd0dfeikh6/development_set.zip [following]
--2020-05-15 05:13:09--  https://www.dropbox.com/s/raw/k89d0cd0dfeikh6/development_set.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbb3f066fe51157bbc0d888f876.dl.dropboxusercontent.com/cd/0/inline/A3s59AYod-ERzaY7R6k4FD63hgNF1QtuuXxQFkBjqij9m3TVxUE54tZD3B2MBmP8vuzxCXMiWU6OPRniY88hE6VCSw15cBYZujY61VPvO4J0UYusTO2nEmF5ok5U_15VzMo/file# [following]
--2020-05-15 05:13:10--  https://ucbb3f066fe51157bbc0d888f876.dl.dropboxusercontent.com/cd/0/inline/A3s59AYod-ERzaY7R6k4FD63hgNF1QtuuXxQFkBjqij9m3TVxUE54tZD3B2MBmP8vuzxCXMiWU6OPRniY88hE6VC

In [30]:
! unzip '/content/development_set.zip'

Archive:  /content/development_set.zip
   creating: development_set/
   creating: development_set/bhart-20100106-lpu/
  inflating: development_set/bhart-20100106-lpu/LICENSE  
   creating: development_set/bhart-20100106-ldp/
  inflating: development_set/bhart-20100106-ldp/LICENSE  
   creating: development_set/bhart-20100106-kab/
  inflating: development_set/bhart-20100106-kab/LICENSE  
   creating: development_set/bhart-20100106-jot/
  inflating: development_set/bhart-20100106-jot/LICENSE  
   creating: development_set/bhart-20100106-joc/
  inflating: development_set/bhart-20100106-joc/LICENSE  
   creating: development_set/bhart-20100106-giy/
  inflating: development_set/bhart-20100106-giy/LICENSE  
   creating: development_set/bhart-20100106-djk/
  inflating: development_set/bhart-20100106-djk/LICENSE  
   creating: development_set/BFG-20120316-bdl/
  inflating: development_set/BFG-20120316-bdl/LICENSE  
   creating: development_set/Berrym-20101102-aiv/
  inflating: development_set/

The below data-sets can be downloaded from here.

  Training corpus : It has been developed from audios taken from ‘on-line VoxForge speech database’ and consists of 5 speech utterances for each speaker, spoken by 34 speakers (i.e, 20-30 seconds/speaker).

  Test corpus: This consists of remaining 5 unseen utterances of the same 34 speakers taken in train corpus.  All audio files are of 10 seconds duration and are sampled at 16000 Hz.


I will strongly recommend you to read our previous post ‘Voice Gender Detection’ as a brief primer about how to work with speech signals are explained there. We have also discussed about extracting a popular speech feature, Mel Frequency Cepstrum Coefficients (MFCCs) previously. A GMM will take as input the MFCCs and derivatives of MFCCs of the training samples of a speaker and will try to learn their distribution, which will be representative of that speaker. A typical speaker identification process can be shown by flow diagram below.

While testing when the speaker of a new voice sample is to be identified, first the 40-dimensional feature (MFCCs + delta MFCC) of the sample will be extracted and then the trained speaker GMM models will be used to calculate the scores of the features for all the models. Speaker model with the maximum score is predicted as the identified speaker of the test speech. Having said that we will go through the python implementation of the following steps:

    40-Dimensional Feature Extraction
    Training Speaker Models.
    Evaluating Performance on test set


Feature Extraction.

We extract 40-dimensional features from speech frames. There are 20 MFCC features and 20 derivatives of MFCC features. The derivatives of MFCCs provides the information of dynamics of MFCCs over the time. It turns out that calculating the delta-MFCC and appending them to the original MFCC features (20-dimenaionl) increases the performance in lot of speech analytics applications. To calculate delta features from MFCCs, we apply the following equation.

The below python functions extracts MFCC features and derives delta coefficients from from audio signal.

In [0]:
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
 
def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while(j <= N):
            if(i-j < 0):
                first = 0
            else:
                first = i-j
            if(i+j > rows -1):
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas
 
def extract_features(audio,rate):
    """extract 20 dim mfcc features from an audio, performs CMS and combines
    delta to make it 40 dim feature vector"""   
 
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True)
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)
    combined = np.hstack((mfcc_feat,delta))
    return combined

2. Training Speaker Models.

As we know, There are 34 distinct speakers in training corpus which are taken from lots of speaker provided by VoxForge. The path of all the audio files (5 per speaker) utilized for training are given in this file. Usually there is a very important step called pre-processing, aslo known as voice activity detection(VAD) which includes noise removal and silence truncation from the audios. I have assumed that there is no requirement of performing VAD here.

In order to build a speaker identification system from the above extracted features, we need to model all the speakers independently now. We employ GMMs for this task.

A Gaussian mixture model is a probabilistic clustering model for representing the presence of sub-populations within an overall population.  The idea of training a GMM is to approximate the probability distribution of a class by a linear combination of ‘k’ Gaussian distributions/clusters, also called the components of the GMM. The likelihood of data points (feature vectors) for a model is given by following equation:



Python’s sklearn.mixture package is used by us to learn a GMM from the features matrix containing the 40 dimensional MFCC and delta-MFCC features. More about sklearn GMM can be read from section 3 of our previous post ‘Voice Gender Detection‘. The following Python code is used to train the GMM speaker models with 16 components. The code is run once for each speaker and train_file is variable which has text filename containing path to all the audios for the respective speaker. Also, you have to create a “speaker_models” directory where all the models will be dumped after training.

In [31]:
! wget https://www.dropbox.com/s/iu12o5sdramh7wo/development_set_enroll.txt?dl=0

--2020-05-15 05:22:32--  https://www.dropbox.com/s/iu12o5sdramh7wo/development_set_enroll.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/iu12o5sdramh7wo/development_set_enroll.txt [following]
--2020-05-15 05:22:32--  https://www.dropbox.com/s/raw/iu12o5sdramh7wo/development_set_enroll.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc93ed550f1294b9f7ef7430b296.dl.dropboxusercontent.com/cd/0/inline/A3sqbFShRNGGDywqn-RnkpUAFgL_27RXDuHj2OajOFS9O0R77CyM5GYp2C9sZgy5YJ84XSsid8t9vr25c7vSxLs2IfonYgGe_peOjmAjilqLRRubB5gL_oOhheGrlSsKJT8/file# [following]
--2020-05-15 05:22:32--  https://uc93ed550f1294b9f7ef7430b296.dl.dropboxusercontent.com/cd/0/inline/A3sqbFShRNGGDywqn-RnkpUAFgL_27RXDuHj2OajOFS9O0R77CyM5GYp2C9sZgy5YJ8

In [43]:
import pickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture as GMM
import warnings
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
warnings.filterwarnings("ignore")
 
#path to training data
source   = "/content/development_set/"  
 
#path where training speakers will be saved
dest = "/content/speaker_models/"
train_file = "/content/development_set_enroll.txt"
file_paths = open(train_file,'r')
 
count = 1
# Extracting features for each speaker (5 files per speakers)
features = np.asarray(())
for path in file_paths:
    path = path.strip()
    #print(path)
    path=path.replace("\\", "/")
    # read the audio
    sr,audio = read(source + path)
 
    # extract 40 dimensional MFCC & delta MFCC features
    vector   = extract_features(audio,sr)
 
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
    # when features of 5 files of speaker are concatenated, then do model training
    if count == 5:
        gmm = GMM(n_components = 16, covariance_type='diag',n_init = 3)
        gmm.fit(features)
 
        # dumping the trained gaussian model
        picklefile = path.split("-")[0]+".gmm"
        pickle.dump(gmm,open(dest + picklefile,'wb'))
        print('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)
        features = np.asarray(())
        count = 0
    count = count + 1

+ modeling completed for speaker: anthonyschaller.gmm  with data point =  (1595, 40)
+ modeling completed for speaker: Apple_Eater.gmm  with data point =  (2077, 40)
+ modeling completed for speaker: Ara.gmm  with data point =  (3644, 40)
+ modeling completed for speaker: argail.gmm  with data point =  (2468, 40)
+ modeling completed for speaker: ariyan.gmm  with data point =  (2506, 40)
+ modeling completed for speaker: arjuan.gmm  with data point =  (2707, 40)
+ modeling completed for speaker: Artem.gmm  with data point =  (2794, 40)
+ modeling completed for speaker: arthur.gmm  with data point =  (3782, 40)
+ modeling completed for speaker: artk.gmm  with data point =  (3369, 40)
+ modeling completed for speaker: arun.gmm  with data point =  (3031, 40)
+ modeling completed for speaker: arvala.gmm  with data point =  (2718, 40)
+ modeling completed for speaker: asalkeld.gmm  with data point =  (3294, 40)
+ modeling completed for speaker: asladic.gmm  with data point =  (1743, 40)
+ m

## 3. Evaluating Performance on Test set.

Test set consists of 5 unseen utterances of trained 34 speakers. The path of all the audio files (5 per speaker) utilized for evaluation are given in this file.

Upon arrival of a test voice sample for speaker identification, we begin by extracting the 40 dimensional for it, with 25 ms frame size and 10 ms overlap between frames. Next we require the log likelihood scores for each frame of the sample, x_1, x_2, ... ,x_i , belonging to each speaker, ie, P(x_i|S_j) (for all j that belongs to S) is to be calculated. The likelihood of the frame being from a particular speaker is calculated by substituting the \mu and \Sigma of that speaker GMM model in likelihood equation shown in previous section. This is done for each of the ‘k’ Gaussian components in the model, and the weighted  sum of the ‘k’ likelihoods from the components is taken as per the weight ‘w ‘ parameter of the model. The logarithm operation when applied on the obtained sum gives us the log likelihood value for the frame. This is repeated for all the frames of the sample and the likelihoods of all the frames are added. The speaker model with highest likelihood score is considered as the identified speaker.

The Python code given below predicts the speaker of the test audio.

In [44]:
! wget https://www.dropbox.com/s/9dg83clhvmm5d40/development_set_test.txt?dl=0

--2020-05-15 05:43:52--  https://www.dropbox.com/s/9dg83clhvmm5d40/development_set_test.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9dg83clhvmm5d40/development_set_test.txt [following]
--2020-05-15 05:43:52--  https://www.dropbox.com/s/raw/9dg83clhvmm5d40/development_set_test.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc1a1c40ae90dc4c4ea33068498.dl.dropboxusercontent.com/cd/0/inline/A3uFGXSXs5w3Aknw-4iRyHGF_kaYOTV14fqi0DZ-tg-UyjQPbqZZ75WMtajsl3wso962nng1fBJqX2hR0jRE5ToOri-_LnOHAIMILLXu4ch0U0-sWmS38L9p7NxgO9ZpHeo/file# [following]
--2020-05-15 05:43:52--  https://ucc1a1c40ae90dc4c4ea33068498.dl.dropboxusercontent.com/cd/0/inline/A3uFGXSXs5w3Aknw-4iRyHGF_kaYOTV14fqi0DZ-tg-UyjQPbqZZ75WMtajsl3wso962nng1f

In [55]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
import warnings
warnings.filterwarnings("ignore")
import time
 
#path to training data
source   = "/content/development_set/"
modelpath = "/content/speaker_models/"
test_file = "/content/development_set_test.txt"
file_paths = open(test_file,'r')
 
gmm_files = [os.path.join(modelpath,fname) for fname in
              os.listdir(modelpath) if fname.endswith('.gmm')]
 
#Load the Gaussian gender Models
models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname
              in gmm_files]
total_file=0
correct=0
wrong=0
# Read the test directory and get the list of test audio files
for path in file_paths:   
 
    path = path.strip()
    #print(path)
    path=path.replace("\\", "/")
    name = path.split('-')[0]
    total_file+=1
    sr,audio = read(source + path)
    vector   = extract_features(audio,sr)
 
    log_likelihood = np.zeros(len(models)) 
 
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
 
    winner = np.argmax(log_likelihood)
    if(name==speakers[winner]):
      correct+=1
    else:
      wrong+=1
    #print("\tdetected as - ", speakers[winner])
    #time.sleep(1.0)
print("total no of files = ",total_file)
print("Correctly identified files = ", correct)
print("Wrong identification = ", wrong)

total no of files =  170
Correctly identified files =  169
Wrong identification =  1


'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

if the above error occurs in pickle.load, it is because it was dumped in r format and was loaded in rb format. That is format mismatch. Load and dump in same format to remove the error

Results and Conclusion

This beginner’s approach performs with an in-set accuracy of 100%, identifying all the 170 speech utterances correctly. There are few reasons for such perfect result.

  The unseen utterances of speakers taken from VoxForge are possibly of same channel or environment.

  The evaluation task in performed on small dataset. Consider a data inflow where you are getting probably some thousands of calls in a day.

  Consider the situation when we have to identify speakers from the set of 1000 speakers.

  In this evaluation, we have not taken out-of-set speakers into account i.e. if the audio is not from any speaker still our system will identify it as one of speakers in trained set depending upon highest likelihood.

  In the real environment, we may get more noisy and unclean data. Speaker identification system needs to be robust.
